In [6]:
import pandas as pd
import numpy as np
import os

In [6]:
# Lee el archivo JSON y carga los datos en un DataFrame de pandas
df = pd.read_json("Datasets/Google Maps/review-estados/review-Alabama/11.json", lines = True)

# Muestra las primeras filas del DataFrame para verificar que se haya cargado correctamente
print(df.head())


        user_id                    name           time  rating  text  pics  \
0  1.116430e+20  Chris and Shea Tolbert  1542773927242       5  None  None   
1  1.179291e+20            clint walker  1549501934566       3  None  None   
2  1.130570e+20                Beth Box  1545440036434       1  None  None   
3  1.051583e+20            Todd Beasley  1530583141411       5  None  None   
4  1.139799e+20           brandi hunter  1483573068099       4  None  None   

   resp                                gmap_id  
0  None  0x88882a58884a0507:0x3bdeae3983aec4dc  
1  None  0x88882a58884a0507:0x3bdeae3983aec4dc  
2  None  0x88882a58884a0507:0x3bdeae3983aec4dc  
3  None  0x88882a58884a0507:0x3bdeae3983aec4dc  
4  None  0x88882a58884a0507:0x3bdeae3983aec4dc  


c:\Users\Mauricio Figueroa\.virtualenvs\Python-env--sma9hmZ\Lib\site-packages\pandas\core\dtypes\astype.py:189: RuntimeWarning: invalid value encountered in cast
  return values.astype(dtype, copy=copy)


In [7]:
# Cada archivo tiene aproximadamente 150 mil registros
len(df)

150000

In [23]:
# La información de cada review esta contenida en las siguientes columnas
df.columns

Index(['user_id', 'name', 'time', 'rating', 'text', 'pics', 'resp', 'gmap_id'], dtype='object')

In [30]:
# Vemos que hay algunos registros (pocos) donde el comercio hace una devolución. Consideramos importante esta devolución
df[~df["resp"].isna()]

,user_id,name,time,rating,text,pics,resp,gmap_id
60,1.019555e+20,MrNiceGuy420,1620252981528,5,"It's quiet , cameras everywhere and a hardee's...",None,"{'time': 1620253626918, 'text': 'Thank you for...",0x88882af73c1dd91f:0x399a5e0c64165330
73,1.061407e+20,Kenneth Smith,1573595680762,5,Awesome and safe place to do your laundry.clea...,None,"{'time': 1573836762445, 'text': 'Thank you! We...",0x88882af73c1dd91f:0x399a5e0c64165330
77,1.183360e+20,Brandy Gipson,1576542079720,5,Its very clean and the washing machines and dr...,None,"{'time': 1576608216452, 'text': 'Thank you Bra...",0x88882af73c1dd91f:0x399a5e0c64165330
97,1.107800e+20,Damien Long,1617727974641,4,👍🏼,None,"{'time': 1617732842952, 'text': 'Thank you for...",0x88882af73c1dd91f:0x399a5e0c64165330
103,1.145796e+20,Kelly Ballam,1616568755311,5,None,None,"{'time': 1617732881857, 'text': 'Thank you for...",0x88882af73c1dd91f:0x399a5e0c64165330
...,...,...,...,...,...,...,...,...
149919,1.086057e+20,Samantha Vines,1556313173819,5,None,None,"{'time': 1556826886268, 'text': 'Thanks so muc...",0x8889cbb390b6d7ab:0x62b42d79d9d55109
149920,1.046784e+20,Carolyn Pearl,1544271470421,5,None,None,"{'time': 1544480389291, 'text': 'Thanks Caroly...",0x8889cbb390b6d7ab:0x62b42d79d9d55109
149921,1.005816e+20,Madison Espy,1527015204203,5,None,None,"{'time': 1527106474830, 'text': 'Thanks-a-mill...",0x8889cbb390b6d7ab:0x62b42d79d9d55109
149926,1.145077e+20,Dale Blakely,1562189862455,5,None,None,"{'time': 1563993083330, 'text': 'Thanks so muc...",0x8889cbb390b6d7ab:0x62b42d79d9d55109


Hacemos una función genérica para recorrer todos los archivos de cada estado y formar un solo DataFrame con la info relevante

In [8]:
def cargar_archivos_json(ruta_base, ruta_destino):

    # Diccionario para llevar el conteo de reviews nulas (review o devolución) por estado
    conteo_nulos = {}

    # Recorre todas las carpetas dentro de la ruta_base
    for estado_folder in os.listdir(ruta_base):

        # Lista para almacenar los df que corresponden a cada json
        lista_dataframes = []

        estado_path = os.path.join(ruta_base, estado_folder)

        # Variable para guardar el nombre del estado
        estado_string = estado_folder.replace("review-", "")
        
        # Inicializa el conteo de nulos para el estado actual
        conteo_nulos[estado_string] = 0

        # Verifica si es una carpeta
        if os.path.isdir(estado_path):
            # Recorre todos los archivos JSON dentro de la carpeta del estado
            for json_file in os.listdir(estado_path):
                if json_file.endswith('.json'):
                    json_path = os.path.join(estado_path, json_file)

                    # Lee el archivo JSON y carga los datos en un DataFrame
                    df = pd.read_json(json_path, lines=True)

                    # Incrementa el conteo de nulos si "text" o "resp" son NaN
                    conteo_nulos[estado_string] += df[["text", "resp"]].isna().all(axis=1).sum()

                    # Limpiamos los nan en la columna de reviews en caso de que la review o la respuesta sean NaN
                    #df = df.dropna(subset = ["text", "resp"], how = "all")

                    # Dejamos las columnas que nos interesan
                    #df = df[["rating", "text", "resp", "gmap_id"]]

                    # Agrega el DataFrame a la lista
                    #lista_dataframes.append(df)

        # Concatena los dataFrames en uno solo
        #review_estado = pd.concat(lista_dataframes, ignore_index = True)

        # Exporta reviews_estado a un archivo CSV
        #csv_path = os.path.join(ruta_destino, f"review-{estado_string}.csv")
        #review_estado.to_csv(csv_path, index=False)

    return conteo_nulos

In [9]:
# Ruta base donde se encuentran las carpetas "review-estados"
ruta_base = "Datasets/Google Maps/review-estados"
ruta_destino = "Datasets/Google Maps/review-estados-clean"

# Llama a la función y almacena el conteo de nulos
conteo_nulos = cargar_archivos_json(ruta_base, ruta_destino)

c:\Users\Mauricio Figueroa\.virtualenvs\Python-env--sma9hmZ\Lib\site-packages\pandas\core\dtypes\astype.py:189: RuntimeWarning: invalid value encountered in cast
  return values.astype(dtype, copy=copy)
c:\Users\Mauricio Figueroa\.virtualenvs\Python-env--sma9hmZ\Lib\site-packages\pandas\core\dtypes\astype.py:189: RuntimeWarning: invalid value encountered in cast
  return values.astype(dtype, copy=copy)
c:\Users\Mauricio Figueroa\.virtualenvs\Python-env--sma9hmZ\Lib\site-packages\pandas\core\dtypes\astype.py:189: RuntimeWarning: invalid value encountered in cast
  return values.astype(dtype, copy=copy)
c:\Users\Mauricio Figueroa\.virtualenvs\Python-env--sma9hmZ\Lib\site-packages\pandas\core\dtypes\astype.py:189: RuntimeWarning: invalid value encountered in cast
  return values.astype(dtype, copy=copy)
c:\Users\Mauricio Figueroa\.virtualenvs\Python-env--sma9hmZ\Lib\site-packages\pandas\core\dtypes\astype.py:189: RuntimeWarning: invalid value encountered in cast
  return values.astype(dty

In [11]:
# Muestra el conteo de nulos por estado, si las columnas text y resp eran nulas
for estado, conteo in conteo_nulos.items():
    print(f"Estado: {estado}, Registros Nulos: {conteo}")

Estado: Alabama, Registros Nulos: 760061
Estado: Alaska, Registros Nulos: 208566
Estado: Arizona, Registros Nulos: 632210
Estado: Arkansas, Registros Nulos: 1011648
Estado: California, Registros Nulos: 1093815
Estado: Colorado, Registros Nulos: 807096
Estado: Connecticut, Registros Nulos: 187694
Estado: Delaware, Registros Nulos: 373253
Estado: District_of_Columbia, Registros Nulos: 255029
Estado: Florida, Registros Nulos: 957433
Estado: Georgia, Registros Nulos: 715159
Estado: Hawaii, Registros Nulos: 615152
Estado: Idaho, Registros Nulos: 795796
Estado: Illinois, Registros Nulos: 829685
Estado: Indiana, Registros Nulos: 912398
Estado: Iowa, Registros Nulos: 1190817
Estado: Kansas, Registros Nulos: 813840
Estado: Kentucky, Registros Nulos: 686014
Estado: Louisiana, Registros Nulos: 651816
Estado: Maine, Registros Nulos: 476105
Estado: Maryland, Registros Nulos: 972803
Estado: Massachusetts, Registros Nulos: 1052584
Estado: Michigan, Registros Nulos: 850459
Estado: Minnesota, Registros

In [18]:
# Registros nulos total
print(f"Registros Nulos total: {sum(conteo_nulos.values())}")

Registros Nulos total: 34613269
